In [161]:
! curl http://web.mta.info/developers/data/nyct/turnstile/turnstile_160402.txt > 160402.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 24.2M    0 24.2M    0     0  3691k      0 --:--:--  0:00:06 --:--:-- 4182k


In [162]:
import csv
from collections import defaultdict
import dateutil.parser

In [163]:
mta = defaultdict(list)
mta2 = defaultdict(list)
mta3 = defaultdict(list)
with open('160402.csv', 'r') as f:
    reader = csv.reader(f)
    reader.next() #skip header row
    for row in reader:
        #read in all of the columns
        ca = row[0]
        unit = row[1]
        scp = row[2]
        station = row[3]
        linename = row[4]
        division = row[5]
        date = row[6]
        time = row[7]
        datetime = dateutil.parser.parse(row[6]+" "+row[7])
        desc = row[8]
        entries = row[9]
        exits = row[10].strip()
                
        #add data to each dictionary
        #use append to add more than one value to one key
        mta[(ca, unit, scp, station)].append([linename, division, date, time, desc, entries, exits])
        mta2[(ca, unit, scp, station)].append([datetime, int(entries)]) #convert entries to numbers from str
        if datetime.hour==0 or datetime.hour==1:
            mta3[ca, unit, scp, station].append([datetime, int(entries)])

In [164]:
for key in mta2: #loop through each key in dictionary
    for i in range(len(mta2[key])-1): #loop through each list of values
        mta2[key][i][1] = abs(mta2[key][i+1][1] - mta2[key][i][1]) #key, list, item in list
    mta2[key].pop() #remove the last date

In [165]:
mta2

defaultdict(list,
            {('J012',
              'R379',
              '00-00-00',
              'KOSCIUSZKO ST'): [[datetime.datetime(2016, 3, 26, 1, 0),
               67], [datetime.datetime(2016, 3, 26, 5, 0), 196], [datetime.datetime(2016, 3, 26, 9, 0),
               359], [datetime.datetime(2016, 3, 26, 13, 0),
               404], [datetime.datetime(2016, 3, 26, 17, 0), 294], [datetime.datetime(2016, 3, 26, 21, 0),
               145], [datetime.datetime(2016, 3, 27, 1, 0),
               43], [datetime.datetime(2016, 3, 27, 5, 0), 144], [datetime.datetime(2016, 3, 27, 9, 0),
               282], [datetime.datetime(2016, 3, 27, 13, 0),
               308], [datetime.datetime(2016, 3, 27, 17, 0), 195], [datetime.datetime(2016, 3, 27, 21, 0),
               117], [datetime.datetime(2016, 3, 28, 1, 0),
               55], [datetime.datetime(2016, 3, 28, 5, 0), 642], [datetime.datetime(2016, 3, 28, 9, 0),
               460], [datetime.datetime(2016, 3, 28, 13, 0),
           

In [166]:
for key in mta3: #loop through each key in dictionary
    for i in range(len(mta3[key])-1): #loop through each list of values
        mta3[key][i][1] = abs(mta3[key][i+1][1] - mta3[key][i][1]) #key, list, item in list
    mta3[key].pop() #remove the last date

In [167]:
mta3

defaultdict(list,
            {('J012',
              'R379',
              '00-00-00',
              'KOSCIUSZKO ST'): [[datetime.datetime(2016, 3, 26, 1, 0),
               1465], [datetime.datetime(2016, 3, 27, 1, 0),
               1089], [datetime.datetime(2016, 3, 28, 1, 0), 1912], [datetime.datetime(2016, 3, 29, 1, 0),
               2079], [datetime.datetime(2016, 3, 30, 1, 0),
               2136], [datetime.datetime(2016, 3, 31, 1, 0), 2171]],
             ('H019',
              'R294',
              '00-06-00',
              'MORGAN AV'): [[datetime.datetime(2016, 3, 26, 0, 0),
               1375], [datetime.datetime(2016, 3, 27, 0, 0),
               889], [datetime.datetime(2016, 3, 28, 0, 0),
               1248], [datetime.datetime(2016, 3, 29, 0, 0),
               1376], [datetime.datetime(2016, 3, 30, 0, 0),
               1452], [datetime.datetime(2016, 3, 31, 0, 0), 1363]],
             ('N131',
              'R383',
              '00-00-01',
              '80 ST')

In [168]:
%matplotlib inline

In [228]:
import pandas as pd
import matplotlib.pyplot as plt

In [229]:
data = pd.read_csv("http://web.mta.info/developers/data/nyct/turnstile/turnstile_160402.txt")

In [230]:
data.columns = [column.strip() for column in data.columns]
data.columns

Index([u'C/A', u'UNIT', u'SCP', u'STATION', u'LINENAME', u'DIVISION', u'DATE',
       u'TIME', u'DESC', u'ENTRIES', u'EXITS'],
      dtype='object')

In [231]:
data.rename(columns={
        "EXITS": "EXIT",
        "ENTRIES": "ENTRY"
    }, inplace=True)

In [232]:
data.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRY,EXIT
0,A002,R051,02-00-00,59 ST,NQR456,BMT,03/26/2016,00:00:00,REGULAR,5595746,1893277
1,A002,R051,02-00-00,59 ST,NQR456,BMT,03/26/2016,04:00:00,REGULAR,5595746,1893282
2,A002,R051,02-00-00,59 ST,NQR456,BMT,03/26/2016,08:00:00,REGULAR,5595746,1893282
3,A002,R051,02-00-00,59 ST,NQR456,BMT,03/26/2016,12:00:00,REGULAR,5595746,1893282
4,A002,R051,02-00-00,59 ST,NQR456,BMT,03/26/2016,16:00:00,REGULAR,5595746,1893282


In [233]:
data = data[['C/A', 'UNIT', 'SCP', 'STATION', 'DATE', 'TIME', 'ENTRY', 'EXIT']]
data['dENTRY'] = 0
data['dEXIT'] = 0
data

,C/A,UNIT,SCP,STATION,DATE,TIME,ENTRY,EXIT,dENTRY,dEXIT
0,A002,R051,02-00-00,59 ST,03/26/2016,00:00:00,5595746,1893277,0,0
1,A002,R051,02-00-00,59 ST,03/26/2016,04:00:00,5595746,1893282,0,0
2,A002,R051,02-00-00,59 ST,03/26/2016,08:00:00,5595746,1893282,0,0
3,A002,R051,02-00-00,59 ST,03/26/2016,12:00:00,5595746,1893282,0,0
4,A002,R051,02-00-00,59 ST,03/26/2016,16:00:00,5595746,1893282,0,0
5,A002,R051,02-00-00,59 ST,03/26/2016,20:00:00,5595746,1893282,0,0
6,A002,R051,02-00-00,59 ST,03/27/2016,00:00:00,5595746,1893282,0,0
7,A002,R051,02-00-00,59 ST,03/27/2016,04:00:00,5595746,1893282,0,0
8,A002,R051,02-00-00,59 ST,03/27/2016,08:00:00,5595746,1893282,0,0
9,A002,R051,02-00-00,59 ST,03/27/2016,12:00:00,5595746,1893282,0,0


In [234]:
data['dENTRY'] = data.ENTRY.shift(-1) - data.ENTRY
data['dEXIT'] = data.EXIT.shift(-1) - data.EXIT
data['dTOTAL'] = data.dENTRY+ data.dEXIT
data

,C/A,UNIT,SCP,STATION,DATE,TIME,ENTRY,EXIT,dENTRY,dEXIT,dTOTAL
0,A002,R051,02-00-00,59 ST,03/26/2016,00:00:00,5595746,1893277,0,5,5
1,A002,R051,02-00-00,59 ST,03/26/2016,04:00:00,5595746,1893282,0,0,0
2,A002,R051,02-00-00,59 ST,03/26/2016,08:00:00,5595746,1893282,0,0,0
3,A002,R051,02-00-00,59 ST,03/26/2016,12:00:00,5595746,1893282,0,0,0
4,A002,R051,02-00-00,59 ST,03/26/2016,16:00:00,5595746,1893282,0,0,0
5,A002,R051,02-00-00,59 ST,03/26/2016,20:00:00,5595746,1893282,0,0,0
6,A002,R051,02-00-00,59 ST,03/27/2016,00:00:00,5595746,1893282,0,0,0
7,A002,R051,02-00-00,59 ST,03/27/2016,04:00:00,5595746,1893282,0,0,0
8,A002,R051,02-00-00,59 ST,03/27/2016,08:00:00,5595746,1893282,0,0,0
9,A002,R051,02-00-00,59 ST,03/27/2016,12:00:00,5595746,1893282,0,0,0


In [235]:
#diff = data['ENTRY'].iloc[1:].values - data['ENTRY'].iloc[:-1].values
#data['DENTRY'][1:] = diff
#diff = data['EXIT'].iloc[:-1].values - data['EXIT'].iloc[1:].values
#data['DEXIT'][1:] = diff

In [236]:
mask = (data['TIME'] == '00:00:00') | (data['TIME']=='01:00:00')

In [237]:
data = data.ix[mask, :]

In [238]:
mask = (data['dENTRY'] >= 0) & (data['dENTRY'] <= 5000) & (data['dEXIT'] >= 0) & (data['dEXIT'] <= 5000)

In [239]:
new_data = data.ix[mask, :]
new_data

,C/A,UNIT,SCP,STATION,DATE,TIME,ENTRY,EXIT,dENTRY,dEXIT,dTOTAL
0,A002,R051,02-00-00,59 ST,03/26/2016,00:00:00,5595746,1893277,0,5,5
6,A002,R051,02-00-00,59 ST,03/27/2016,00:00:00,5595746,1893282,0,0,0
12,A002,R051,02-00-00,59 ST,03/28/2016,00:00:00,5595746,1893282,0,0,0
18,A002,R051,02-00-00,59 ST,03/29/2016,00:00:00,5596896,1893974,9,1,10
24,A002,R051,02-00-00,59 ST,03/30/2016,00:00:00,5598003,1894856,7,3,10
30,A002,R051,02-00-00,59 ST,03/31/2016,00:00:00,5599165,1895684,0,0,0
37,A002,R051,02-00-00,59 ST,04/01/2016,00:00:00,5599639,1896005,11,10,21
43,A002,R051,02-00-01,59 ST,03/26/2016,00:00:00,5143982,1127500,0,6,6
49,A002,R051,02-00-01,59 ST,03/27/2016,00:00:00,5143982,1127506,0,0,0
55,A002,R051,02-00-01,59 ST,03/28/2016,00:00:00,5143982,1127506,0,0,0


In [240]:
datasum = new_data.groupby([new_data.DATE,new_data.STATION]).sum()
datasum

ENTRY        EXIT  dENTRY  dEXIT  dTOTAL
DATE       STATION                                                       
03/26/2016 1 AV             1409009188  1462205971    1848   1050    2898
           103 ST            152097386   594017455     288    380     668
           103 ST-CORONA      62520085    59867349     238   1266    1504
           104 ST             16816556    15095950      36    114     150
           110 ST             26187023    26772943     280    682     962
           111 ST            111757071    59025097     168    701     869
           116 ST            436233139   446180695     553    585    1138
           116 ST-COLUMBIA  1210451849   256326814     138    163     301
           121 ST              7985059     6603030      80     51     131
           125 ST            643817979   523438192    1878   2429    4307
           135 ST            240740367    46006358     547    495    1042
           137 ST CITY COL    52618109    41609282     166    494     660
           138/GRAND CONC    996333794   962170261      64    125     189
           14 ST             589810971   191535105    2788   1124    3912
           14 ST-UNION SQ   1415380109   500640412    3640   1143    4783
           145 ST           1476187007  2081939631     701   2708    3409
           149/GRAND CONC    527580221   204389742     359    447     806
           155 ST             31241649    23827344     106    265     371
           157 ST             30879012    19125107     147    366     513
           163 ST-AMSTERDM    16964810    11379831      48    151     199
           167 ST           1833384052   891209976     311    972    1283
           168 ST            137037379    59757156     385    698    1083
           169 ST             24797346    26385081      53    352     405
           170 ST           1302877492  2361608165     274    608     882
           174 ST             22035313    14796796      87    274     361
           174-175 STS        14956395    10063490      89     93     182
           175 ST            154535231    19309114     106    435     541
           176 ST             12286038     8393592     120    181     301
           18 AV            1735510843   862441549     103    362     465
           18 ST             223274019   748739152     243    128     371
...                                ...         ...     ...    ...     ...
04/01/2016 ST LAWRENCE AV     12168512     9256040      63    311     374
           ST. GEORGE         29549286       96619     234      0     234
           STEINWAY ST        47495436    35435765     242    296     538
           STERLING ST        22931589    16054449      52    154     206
           SUTPHIN BLVD       18231961     9827746     106     72     178
           SUTPHIN-ARCHER    150538345    76108513     575    734    1309
           SUTTER AV          10961550    15013390     104    132     236
           SUTTER AV-RUTLD    28268372    16470096      98    265     363
           TIMES SQ-42 ST   3825616648  3081404086    3681   1086    4767
           TREMONT AV        129618586    11185629     224    289     513
           UNION ST           20192784    21278003      98    121     219
           UTICA AV          103661701    50682123     254    325     579
           V.CORTLANDT PK     22542898     7751982      47     63     110
           VAN SICLEN AV      30285735    21136550     104    202     306
           VAN SICLEN AVE     13440995    13925679      76     99     175
           VERNON-JACKSON     32926930    22207471     172    303     475
           W 4 ST-WASH SQ    805268678   897511454    1364    497    1861
           W 8 ST-AQUARIUM    10943051    11987003       9     20      29
           WAKEFIELD/241      16420450    16980326     168    185     353
           WALL ST           109245985    95851020     348    178     526
           WEST FARMS SQ     438638202    73183778     208    185     393
           WESTCHESTER SQ     2353412

In [183]:
plt.figure(figsize=(10,3))
plt.plot(dates,entries)

NameError: name 'dates' is not defined

In [ ]:
pd.__version__

In [ ]:
data